# Docker部署模型

## 拉取 Docker 镜像

```shell
>>> docker pull tensorflow/serving
```

## 在运行的容器内运行 ModelServer

```shell
>>> tensorflow_model_server --port=8500 --rest_api_port=8501 \
  --model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME}
```
- `gRPC` 端口为 8500
- `REST API` 端口为 8501
- 环境变量 `MODEL_NAME`，默认值为 `model`，标识模型的名称
- 环境变量 `MODEL_BASE_PATH`，默认值 `/models`，标识模型文件存放的路径

## 通过docker运行ModelServer

```shell
>>> docker run -p 8501:8501 \
  --mount type=bind,source=/path/to/my_model/,target=/models/my_model \
  -e MODEL_NAME=my_model -t tensorflow/serving
```
- `-p 8501:8501` 
    - 将容器的 8501 端口绑定到宿主机的 8501 端口
- `--mount type=bind,source=/path/to/my_model/,target=/models/my_model` 
    - 将存放在本地目录`source=/path/to/my_model/`中的名为 `my_model` 的模型，挂载到容器的的路径上`target=/models/my_model`
    
----    
    
    
运行该命令，会启动 `serving` 的容器，并在容器内部运行:
```shell
    >>> tensorflow_model_server --port=8500 --rest_api_port=8501 \
       --model_name=my_model --model_base_path=/models/my_model
```

## 其它参数

`tensorflow_model_server` 支持其它一些参数，例如提供模型配置文件，而不是模型名称：
```shell
>>> docker run -p 8500:8500 -p 8501:8501 \
  --mount type=bind,source=/path/to/my_model/,target=/models/my_model \
  --mount type=bind,source=/path/to/my/models.config,target=/models/models.config \
  -t tensorflow/serving --model_config_file=/models/models.config
```

```shell
# 查看正在运行的容器
hahaha@carpediem:~$ docker container ls
CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS              
9986cca3b23d        tensorflow/serving   "/usr/bin/tf_serving…"   35 minutes ago      Up 35 minutes       

# 指定容器ID，进入容器内，执行 bash 命令
hahaha@carpediem:~$ docker exec -it 9986cca3b23d /bin/bash

# 在容器内查看 tensorflow_model_server 命令
root@9986cca3b23d:/# tensorflow_model_server --help
usage: tensorflow_model_server
Flags:
	--port=8500                      	int32	Port to listen on for gRPC API
	--grpc_socket_path=""            	string	If non-empty, listen to a UNIX socket for gRPC API on the 
	--rest_api_port=0                	int32	Port to listen on for HTTP/REST API. If set to zero HTTP/REST 
```

# 将模型封装成 docker

## 启动镜像，作为守护进程

```shell
>>> docker run -d --name serving_base tensorflow/serving
```

## 将保存的模型复制到容器内

```shell
>>> docker cp models/<my model> serving_base:/models/<my model>
```

## 重命名封装的模型容器

```shell
>>> docker commit --change "ENV MODEL_NAME <my model>" serving_base <my container>
```

停止守护进程容器 `>>> docker kill serving_base`，剩下的名为 `<my container>` 的容器即为封装的模型容器

# 保存模型

- 保存通过 keras 训练得到的模型 `model`，需要指定输入输出参数`inputs, outputs`

```python
tf.saved_model.simple_save(
    keras.backend.get_session(),
    directory_path,
    inputs={"input_image": model.input},
    outputs={i.name: i for i in model.outputs},
)
```
-----------------------
- 查看模型文件夹内容，模型被保存为 `.pb` 文件：
     
```
hahah7@carpediem:/path/to/my_model$ tree
.
├── assets
│   └── foo.txt
├── saved_model.pb
└── variables
    ├── variables.data-00000-of-00001
    └── variables.index

2 directories, 4 files
```




- 查看模型内部信息：

```shell
hahah7@carpediem:/path/to/my_model$ saved_model_cli show --dir . --al
MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['classify_x_to_y']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_STRING
        shape: unknown_rank
        name: tf_example:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y:0
  Method name is: tensorflow/serving/classify
```

# 调用模型

命令行：
```
>>> curl -d '{"instances": [1.0, 2.0, 5.0]}' -X POST http://localhost:8501/v1/models/half_plus_two:predict
```            
- POST 的数据必须满足 `tf.saved_model` 时的格式：`inputs={"instances": model.input}   --> {"instances": [1.0, 2.0, 5.0]}`
- URL 为 `http://localhost:8501/v1/models/<model_name>:predict`，其中 `<model_name>` 为启动镜像时的模型名称

python:
```python
x = np.array([1.0, 2, 3])
data = json.dump({"instances": x})
res = requests.post("http://localhost:8501/v1/models/half_plus_two:predict",
                    data=data)
```

# 示例

## 创建模型 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

my_devices = tf.config.experimental.list_physical_devices()
my_devices

No GPU found


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [3]:
(X_train_full, y_train_full), (X_test,
                               y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

In [4]:
X_train.shape, X_valid.shape, y_valid.shape

((55000, 28, 28, 1), (5000, 28, 28, 1), (5000,))

In [5]:
np.random.seed(10)
tf.random.set_seed(42)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28, 1]),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 1s 577us/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 1s 517us/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 1s 527us/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 1s 536us/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 1s 524us/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 1s 524us/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 1s 536us/step - loss: 0.2161 - accuracy: 0.9400 - val_loss: 0.1970 -

In [6]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [7]:
import os
model_version = "0001"
model_name = "../../H/models/my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'../../H/models/my_mnist_model/0001'

## 本地模型文件详情

In [8]:
!rm -rf model_name

In [9]:
# 保存模型
tf.saved_model.save(model, model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../../H/models/my_mnist_model/0001/assets


In [10]:
# 模型文件
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

                my_mnist_model/
                    0001/
                        saved_model.pb
                        assets/
                        variables/
                            variables.data-00001-of-00002
                            variables.data-00000-of-00001
                            variables.index
                            variables.data-00000-of-00002


In [11]:
# 显示模型详情
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
'serve'


In [12]:
!saved_model_cli show --dir {model_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [13]:
!saved_model_cli show --dir {model_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [14]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '

## 调用模型

In [15]:
# 本地文件
np.save("../datasets/my_mnist_tests.npy", X_new)

In [19]:
model.input_names

['flatten_input']

In [20]:
model.output_names

['dense_1']

In [16]:
input_name = model.input_names[0]
input_name

'flatten_input'

In [17]:
# 调用本地模型
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=../datasets/my_mnist_tests.npy

2020-06-13 18:37:29.540254: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-13 18:37:29.542962: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-06-13 18:37:29.542999: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: carpediem
2020-06-13 18:37:29.543005: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: carpediem
2020-06-13 18:37:29.543073: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 440.48.2
2020-06-13 18:37:29.543118: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 440.48.2
2020-06-13 18:37:29.543123: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 440.48.2
2020-06-13 18:37:29.543294: I tensorflow/core/platform/cpu_featu

In [21]:
np.round([[
    1.1739199e-04, 1.1239604e-07, 6.0210604e-04, 2.0804715e-03, 2.5779348e-06,
    6.4079795e-05, 2.7411186e-08, 9.9669880e-01, 3.9654213e-05, 3.9471846e-04
],
          [
              1.2294615e-03, 2.9207937e-05, 9.8599273e-01, 9.6755642e-03,
              8.8930705e-08, 2.9156188e-04, 1.5831805e-03, 1.1311053e-09,
              1.1980456e-03, 1.1113169e-07
          ],
          [
              6.4066830e-05, 9.6359509e-01, 9.0598064e-03, 2.9872139e-03,
              5.9552520e-04, 3.7478798e-03, 2.5074568e-03, 1.1462728e-02,
              5.5553433e-03, 4.2495009e-04
          ]], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.96, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.01, 0.  ]])

## 部署模型

In [21]:
model_path = os.path.abspath('../../H/models/my_mnist_model')
model_path

'/home/yangbin7/H/models/my_mnist_model'

In [22]:
# 运行模型
!docker run -it --rm -p 8500:8500 -p 8501:8501 \
        -v "$model_path:/models/my_mnist_model" \
        -e MODEL_NAME=my_mnist_model \
        -d tensorflow/serving

3e97864a384d5d2d0661b038824af45f358e38ed37fd9f75b8d8320a32ca02f3


In [23]:
!docker ps

CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS              PORTS                              NAMES
3e97864a384d        tensorflow/serving   "/usr/bin/tf_serving…"   3 seconds ago       Up 2 seconds        0.0.0.0:8500-8501->8500-8501/tcp   upbeat_knuth


## REST API

In [29]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(), # 为什么是这种形式
})

In [30]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [31]:
response

{'predictions': [[0.000113945083,
   1.51165395e-07,
   0.00097856042,
   0.00277804374,
   3.7648972e-06,
   7.63042772e-05,
   3.89179782e-08,
   0.99556762,
   5.23388844e-05,
   0.000429277687],
  [0.000816865184,
   3.53646938e-05,
   0.988218,
   0.00707711279,
   1.28269079e-07,
   0.000233427039,
   0.0025804257,
   9.64154867e-10,
   0.00103865331,
   8.68936567e-08],
  [4.47355778e-05,
   0.970272243,
   0.00908343308,
   0.00227006385,
   0.000486535311,
   0.00287033687,
   0.00227623642,
   0.00836951472,
   0.00402864302,
   0.000298208324]]}

In [32]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

## gRPC API

In [35]:
from tensorflow_serving_apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

ModuleNotFoundError: No module named 'tensorflow_serving_apis'

In [ ]:
import grpc
from tensorflow_serving_apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [ ]:
response

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
y_proba.round(2)

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

In [31]:
class TextCnn(tf.keras.Model):
    def __init__(self,
                 maxlen,
                 max_features,
                 class_num,
                 embedding_dims,
                 embeddings_matrix,
                 kernel_sizes=[1, 2, 3, 4, 5],
                 last_activation='sigmoid'):  
        super(TextCnn, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.kernel_sizes = kernel_sizes
        self.class_num = class_num
        self.last_activation = last_activation
        self.embeddings_matrix = embeddings_matrix
        self.embedding = keras.layers.Embedding(
            self.max_features,
            self.embedding_dims,
            weights=[embeddings_matrix],
            input_length=self.maxlen)  # 重点：weights预训练的词向量
        self.convs = []
        self.max_poolings = []
        for kernel_size in self.kernel_sizes:
            self.convs.append(keras.layers.Conv1D(128, kernel_size))
            self.max_poolings.append(keras.layers.GlobalMaxPooling1D())
        self.classifier = keras.layers.Dense(self.class_num,
                                             activation=self.last_activation)
        self.dropout = keras.layers.Dropout(0.15)  #百分之20的神经元不工作
        self.norma = keras.layers.LayerNormalization()
        self.act = keras.layers.Activation('relu')
        self.softmax = keras.layers.Softmax()

    def call(self, inputs):
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError(
                'The maxlen of inputs of TextCNN must be %d, but now is %d' %
                (self.maxlen, inputs.get_shape()[1]))
        embedding = self.embedding(inputs)
        convs = []
        for i in range(len(self.kernel_sizes)):
            c = self.convs[i](embedding)
            c = self.norma(c)
            c = self.act(c)
            c = self.max_poolings[i](c)
            convs.append(c)
        x = keras.layers.Concatenate()(convs)
        x_1 = self.dropout(x)
        output = self.classifier(x_1)
        score = self.softmax(output)  #将全链接层的输出，进行softmax运算，得到每个类的概率
        return score

In [32]:
from tensorflow import keras
word_matrix = tf.random.uniform([30, 50])
model = TextCnn(20, 30, 4, 50, word_matrix)

In [33]:
inputs = np.random.randint(2, 10, (3, 20))
model(inputs)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0.26742765, 0.26718795, 0.27480498, 0.1905794 ],
       [0.26635906, 0.2439773 , 0.28277802, 0.20688564],
       [0.2495458 , 0.27974066, 0.28159958, 0.18911393]], dtype=float32)>

In [34]:
tf.saved_model.save(model, 'hhhhh')

INFO:tensorflow:Assets written to: hhhhh/assets


In [35]:
model_path = "hhhhh/"
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
'serve'


In [36]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 20)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
    Optio

In [29]:
reshape = tf.keras.layers.Reshape((-1, 10))

In [30]:
reshape(inputs)

<tf.Tensor: shape=(3, 2, 10), dtype=int64, numpy=
array([[[5, 3, 6, 2, 4, 4, 4, 3, 7, 4],
        [3, 7, 9, 8, 5, 4, 2, 9, 3, 3]],

       [[4, 4, 7, 9, 5, 4, 6, 4, 5, 7],
        [5, 3, 2, 6, 8, 5, 9, 8, 8, 5]],

       [[7, 3, 6, 7, 3, 3, 8, 7, 5, 6],
        [2, 4, 9, 2, 7, 2, 3, 8, 4, 4]]])>

In [1]:
import tensorflow as tf

In [5]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(100, activation='relu')
        self.dense2 = tf.keras.layers.Dense(6, activation='softmax')

    def call(self, inp):
        # 当使用 (inp.shape[0],-1) 模型就无法保存成功
        # 当使用 [tf.shape(inp)[0], -1] 成功
        x = tf.reshape(inp, [tf.shape(inp)[0], -1])
        x = self.dense1(x)
        y = self.dense2(x)
        return y

In [6]:
model = MyModel()
inp = tf.random.uniform([2, 28, 28, 1])
model(inp)

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.10305609, 0.13621774, 0.13960676, 0.3097873 , 0.26117367,
        0.0501584 ],
       [0.17174944, 0.14442973, 0.15293509, 0.3334627 , 0.16186215,
        0.03556088]], dtype=float32)>

In [7]:
tf.saved_model.save(model, 'hhhh')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: hhhh/assets


In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(100, activation='relu')
        self.dense2 = tf.keras.layers.Dense(6, activation='softmax')

    def call(self, inp):
        # 当使用 (inp.shape[0],-1) 模型就无法保存成功
        # 当使用 [tf.shape(inp)[0], -1] 成功
        x = tf.reshape(inp, [tf.shape(inp)[0], -1])
        x = self.dense1(x)
        y = self.dense2(x)
        return y

In [28]:
class MyModel(tf.keras.Model):
    def __init__(self, aspect_input, num_class):
        super(MyModel, self).__init__()
        self.aspect_input = aspect_input
        self.num_aspect, self.aspect_len = aspect_input.shape

        self.num_class = num_class

        self.embed = tf.keras.layers.Embedding(10000, 56)
        self.dense1 = tf.keras.layers.Dense(100, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_class, activation='softmax')

    def _get_aspect(self):
        aspect_embed = self.embed(self.aspect_input)
        return aspect_embed

    def call(self, inp):  # batch,seq_len
        batch = tf.shape(inp)[0]
        word_embed = self.embed(inp)  # batch,seq_len,embed_size
        word_embed = tf.reduce_sum(word_embed, axis=1)  # batch,embed_size
        word_embed = tf.tile(tf.expand_dims(word_embed, axis=1),
                             multiples=[1, self.num_aspect, 1])
        print(word_embed.shape)
        # batch,num_aspect,embed_size

        aspect_embed = self._get_aspect()  # num_aspect,aspect_len,embed_size
        aspect_embed = tf.reduce_mean(aspect_embed,
                                      axis=1)  # num_aspect,embed_size
        aspect_embed = tf.tile(tf.expand_dims(aspect_embed, axis=0),
                               multiples=[batch, 1, 1])
        print(aspect_embed.shape)
        # batch,num_aspect, embed_size
        
        embed = tf.concat([word_embed,aspect_embed], axis=-1)
        print(embed.shape)
        # batch,num_aspect,2*embed_size
        
        x = self.dense1(embed)
        y = self.dense2(embed)

        return y

In [29]:
import numpy as np

In [30]:
aspect = tf.constant(np.random.randint(1, 100, (7, 4)), dtype=tf.int32)
aspect                     

<tf.Tensor: shape=(7, 4), dtype=int32, numpy=
array([[50, 21,  3, 44],
       [41, 46,  2, 31],
       [77, 29, 62, 52],
       [81, 89, 87, 78],
       [57, 78, 90,  3],
       [25, 16,  3,  1],
       [21, 11, 60, 38]], dtype=int32)>

In [31]:
inp = tf.constant(np.random.randint(1, 1000, (2, 9)), dtype=tf.int32)
inp

<tf.Tensor: shape=(2, 9), dtype=int32, numpy=
array([[168, 990, 338, 376, 620, 444, 128, 230, 121],
       [ 64, 293,  38,  86, 524, 476, 543, 853, 988]], dtype=int32)>

In [32]:
my_model = MyModel(aspect, 4)

In [35]:
my_model(inp)

(2, 7, 56)
(2, 7, 56)
(2, 7, 112)


<tf.Tensor: shape=(2, 7, 4), dtype=float32, numpy=
array([[[0.2535123 , 0.25352374, 0.24321787, 0.24974607],
        [0.2602636 , 0.25252223, 0.24250811, 0.24470611],
        [0.2635415 , 0.25121826, 0.24079615, 0.24444407],
        [0.25485468, 0.25221926, 0.24111736, 0.25180873],
        [0.25300932, 0.24960595, 0.24246114, 0.25492358],
        [0.2552147 , 0.2494412 , 0.24522544, 0.25011867],
        [0.25443554, 0.2559735 , 0.23960853, 0.2499824 ]],

       [[0.25909325, 0.23844846, 0.2584526 , 0.24400571],
        [0.26591882, 0.23744014, 0.25762632, 0.23901473],
        [0.26925492, 0.2362026 , 0.25579524, 0.23874725],
        [0.2604839 , 0.23723859, 0.2562389 , 0.24603862],
        [0.25856486, 0.23475061, 0.25763413, 0.24905041],
        [0.2607295 , 0.23451546, 0.26048237, 0.24427265],
        [0.26012966, 0.24083848, 0.25470805, 0.24432382]]], dtype=float32)>

In [36]:
tf.saved_model.save(my_model, 'hhhh')

(None, 7, 56)
(None, 7, 56)
(None, 7, 112)
(None, 7, 56)
(None, 7, 56)
(None, 7, 112)
INFO:tensorflow:Assets written to: hhhh/assets


In [38]:
!saved_model_cli show --dir 'hhhh' --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 9)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7, 4)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
    Opt

# 不使用 tf.function 时保存没有问题